In [1]:
# Set current directory as working directory
import os
import sys
import csv
import io
from pathlib import Path

path_main = str(Path(os.path.dirname(os.path.abspath(''))).parents[0])
print(f"Current main owrking directory: {path_main}")
os.chdir(path_main)

# Imporant statistical tools packages
import pandas as pd
import scipy.stats as stats

# Import visualization packages
import matplotlib.pyplot as plt
import seaborn as sns

Current main owrking directory: q:\Projects\Variable_resolution\Programming\maskrcnn-combined-build


### Static variables

In [2]:
# Choose parent directory"
csv_file_1 = os.path.join(path_main, "EXPERIMENTS/bin_eval_per_obj_type_ann_norm_small_med/evaluations/equiconst_pretrained_resnet_norm_0.1_340_an_ylarge_nfp/eval_across_bins_on_104.csv")
csv_file_2 = os.path.join(path_main, "EXPERIMENTS/bin_eval_per_obj_type_ann_norm_small_med/evaluations/var_res_pretrained_resnet_norm_0.1_340_an_ylarge_nfp/eval_across_bins_on_104.csv")

csv_file_1

'q:\\Projects\\Variable_resolution\\Programming\\maskrcnn-combined-build\\EXPERIMENTS/bin_eval_per_obj_type_ann_norm_small_med/evaluations/equiconst_pretrained_resnet_norm_0.1_340_an_ylarge_nfp/eval_across_bins_on_104.csv'

### Auxiliary functions

In [18]:
def split_combined_csv_into_n_trials_dataframes(filepath: str):

    dataframes = []
    rowData = []

    # Function to check whether a line is empty
    def is_empty_line(line):
        return all(c in (',', ' ') for c in line)

    # Open the csv and read line by line
    with open(filepath, 'r') as file:
        for line in file:
            stripped_line = line.strip()

            # Check if the line is 'empty'
            if is_empty_line(stripped_line) or stripped_line == '':
                if rowData:
                    df = pd.read_csv(io.StringIO("\n".join(rowData)))
                    dataframes.append(df)
                    rowData = []  # Clear the row data
            else:
                rowData.append(stripped_line)
                
    # Add last batch of data to dataframe list, if it's non-empty
    if rowData:
        df = pd.read_csv(io.StringIO("\n".join(rowData)))
        dataframes.append(df)

    return dataframes

def generate_two_dim_vector_of_metrics(dataframe_list_1, dataframe_list_2,
 metric_name: str, bin_lower_thresh: float):
    '''
    Generates and returns two lists of the given metric, where each value is extracted from its coresponding trial dataframe
    in sequntial order.
    '''
    df_1_data = []
    df_2_data = []

    for df_1, df_2 in zip(dataframe_list_1, dataframe_list_2):
        if df_1.shape != df_2.shape:
            raise ValueError("Dataframes must have the same shape.")

        selected_row = df_1.loc[df_1['lower_bin_thresh'] == bin_lower_thresh, metric_name]
        df_1_data.append(selected_row.to_list()[0])

        selected_row = df_2.loc[df_2['lower_bin_thresh'] == bin_lower_thresh, metric_name]
        df_2_data.append(selected_row.to_list()[0])

    return df_1_data, df_2_data
        


### Load the data

In [12]:
model_1_trials_dfs = split_combined_csv_into_n_trials_dataframes(csv_file_1)
model_2_trials_dfs = split_combined_csv_into_n_trials_dataframes(csv_file_2)

## Statistical tests

### Two-sampled t-test (BBOX AP)

In [51]:
metric = "bbox_AP"
bin_lower_thresh = 0.0

selected_metric_model_1, selected_metric_model_2 = generate_two_dim_vector_of_metrics(model_1_trials_dfs, model_2_trials_dfs, metric, bin_lower_thresh)

In [52]:
t_statistic, p_value = stats.ttest_rel(selected_metric_model_1, selected_metric_model_2, alternative = 'greater')
print("---RELATED-SAMPLES---")
print("t statistic:", t_statistic)
print("p value:", p_value)

t_statistic, p_value = stats.ttest_rel(selected_metric_model_1, selected_metric_model_2, alternative = 'greater')
print("---RELATED-SAMPLES---")
print("t statistic:", t_statistic)
print("p value:", p_value)

t statistic: 2.074920069605781
p value: 0.05331715705824098
